# Project Kaggle

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model.perceptron import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

X = np.genfromtxt("data/x_train.txt")
Y = np.genfromtxt("data/y_train.txt")

X_submission = np.genfromtxt("data/x_test.txt")

xtr, xte, ytr, yte = ml.splitData(X, Y, 0.75)

n_folds = 2

# KNN Classifier

In [4]:
k = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
# after testing ks, found that 3 works the best
# n_neighs = 3

X, Y = ml.shuffleData(X,Y)


knn = KNeighborsClassifier(n_neighbors = 7)
knn.fit(X, Y)
yha = knn.predict_proba(X_submission)[0:,1]
print(i, "AUC of Training Data", metrics.roc_auc_score(Y, yha))


    
print("Saving Results.")
tmp = np.vstack([range(0, len(yha)), yha]).T
np.savetxt(fname='knn.csv', X=tmp, fmt='%d, %0.2f', header='ID,Prob1', comments='')

9 AUC of Training Data 0.500547261103
Saving Results.


# Decision Tree

In [27]:
for i in range(1, 20):
    dt_stump = DecisionTreeClassifier(max_depth=i) #, min_samples_leaf=1) AUC :  0.597725685282
    dt_stump.fit(xtr, ytr)
    print(i, " AUC : ",  metrics.roc_auc_score(yte, dt_stump.predict(xte)))

1  AUC :  0.606374195455
2  AUC :  0.587166766609
3  AUC :  0.578756826554
4  AUC :  0.577866129551
5  AUC :  0.595884331785
6  AUC :  0.590903556653
7  AUC :  0.602237887452
8  AUC :  0.602845635947
9  AUC :  0.605458793316
10  AUC :  0.609862811012
11  AUC :  0.614330048043
12  AUC :  0.620181211266
13  AUC :  0.624873596984
14  AUC :  0.626535857967
15  AUC :  0.63212966562
16  AUC :  0.632743946054
17  AUC :  0.637610191184
18  AUC :  0.638471458804
19  AUC :  0.641209535953


# Logistic Regression Classifier

In [22]:
from sklearn.linear_model import LogisticRegression

LogReg = LogisticRegression()
LogReg.fit(X, Y)

logreg_yha = LogReg.predict(X)

print("Log Reg  roc : ", metrics.roc_auc_score(Y, logreg_yha))

Log Reg  roc :  0.573358700905


# AdaBoost Classifier

In [3]:
print("starting")
for i in range(1,10):
    ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=i))
    ada.fit(X, Y)

    ada_yha = ada.predict(X)

    ada_auc = metrics.roc_auc_score(Y, ada_yha)

    print(i, " Ada  roc : ", ada_auc)

starting
1  Ada  roc :  0.599082347648
2  Ada  roc :  0.610880367175
3  Ada  roc :  0.625808746306
4  Ada  roc :  0.645105898815
5  Ada  roc :  0.657367091295
6  Ada  roc :  0.676674477783
7  Ada  roc :  0.704768180203
8  Ada  roc :  0.73223079933
9  Ada  roc :  0.767757239354


# Gradient Boost Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

for i in range(1,21):
    gbc = GradientBoostingClassifier(max_depth=i)
    gbc.fit(xtr, ytr)
    gbc_yha = gbc.predict(xte)

    gbc_auc = metrics.roc_auc_score(yte, gbc_yha)

    print(i, " gbc  roc : ", gbc_auc)


# Total

In [ ]:
tot_xtr = np.zeros((ada_yha.shape[0], 3))

tot_xtr[:,0] = knn_yha
tot_xtr[:,1] = ada_yha
tot_xtr[:,2] = gbc_yha

y_submission = np.average(tot_xtr, axis=1, weights=[knn_auc, ada_auc, gbc_auc])

tot_acc = metrics.roc_auc_score(yte, y_submission)

print("all  roc : ", tot_acc)

print("Saving Results.")
tmp = np.vstack([range(0, len(y_submission)), y_submission]).T
np.savetxt(fname='submission.csv', X=tmp, fmt='%d, %0.2f', header='ID,Prob1', comments='')

In [21]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier

np.random.seed(0)  # seed to shuffle the train set


verbose = True
shuffle = False

skf = list(StratifiedKFold(ytr, n_folds))

clfs = [KNeighborsClassifier(n_neighbors=7),
        AdaBoostClassifier(),
        GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=50)]

print("Creating train and test sets for blending.")

dataset_blend_train = np.zeros((xtr.shape[0], len(clfs)))
dataset_blend_test = np.zeros((xte.shape[0], len(clfs)))

for j, clf in enumerate(clfs):
    print(j, clf)
    dataset_blend_test_j = np.zeros((xte.shape[0], len(skf)))
    for i, (train, test) in enumerate(skf):
        print("Fold ", i)
        X_train = xtr[train]
        Y_train = ytr[train]
        X_test = xtr[test]
        Y_test = ytr[test]
        clf.fit(X_train, Y_train)
        y_submission = clf.predict_proba(X_test)[:, 1]
        print("  AUC = ", metrics.roc_auc_score(Y_test, y_submission))
        dataset_blend_train[test, j] = y_submission
        dataset_blend_test_j[:, i] = clf.predict_proba(xte)[:, 1]
    dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)
    print("        Total AUC ", metrics.roc_auc_score(yte, dataset_blend_test[:,j]))

print()
print("Blending.")
clf = LogisticRegression()
clf.fit(dataset_blend_train, ytr)
y_submission = clf.predict_proba(dataset_blend_test)[:, 1]


print("TOT  roc : ", metrics.roc_auc_score(yte, y_submission))

Creating train and test sets for blending.
0 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='uniform')
Fold  0
  AUC =  0.696541623114
Fold  1
  AUC =  0.693617349124
        Total AUC  0.728292177567
1 AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)
Fold  0
  AUC =  0.675343785011
Fold  1
  AUC =  0.679155974813
        Total AUC  0.679207812092
2 GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=6,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=50,
              presort='auto', random_state=None, subsample=0.5, verbose=0,
              warm_start=F

In [ ]:
print("Saving Results.")
tmp = np.vstack([range(0, len(dataset_blend_train)), dataset_blend_train]).T
np.savetxt(fname='train.csv', X=tmp) #, fmt='%d, %0.2f', header='ID,Prob1', comments=''



In [ ]:
print("Linear stretch of predictions to [0,1]")
y_submission = (y_submission - y_submission.min()) / (y_submission.max() - y_submission.min())

print("Saving Results.")
tmp = np.vstack([range(0, len(y_submission)), y_submission]).T
np.savetxt(fname='submission.csv', X=tmp, fmt='%d, %0.2f', header='ID,Prob1', comments='')